In [ ]:
import cv2
import numpy as np
import pandas as pd
from keras.layers import Conv2D, Flatten, Dense, LSTM, TimeDistributed, \
    MaxPooling2D
from keras.models import Sequential

IMAGE_SIZE = (64, 64, 3)

def extract_training_data(filename, csv_filename, image_size=(64, 64, 3)):
    cap = cv2.VideoCapture(filename)
    labels = pd.read_csv(csv_filename, sep="\t", header=0)
        
    frame_counter = 0
    processed_frames = []

    training_images = []
    training_label_ids = []
    while True:
        frame_counter += 1
        result, frame = cap.read() 
        if result and frame_counter % 12 == 0:
            # cv2.imshow("img", frame)
            frame = frame / 255
            resized = cv2.resize(frame, image_size[:2])
            processed_frames.append(resized)
            if len(processed_frames) >= 4:
                # cv2.imshow('frame', resized)

                # stacked_image = np.concatenate(processed_frames, axis=2)
                # training_labels.append(labels[frame_counter])
                training_images.append(processed_frames.copy())
                training_label_ids.append(frame_counter)
                processed_frames.pop(0)

        if cv2.waitKey(1) & 0xFF == ord('q') or not result:
            break

    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()

    training_images = np.array(training_images)
    training_labels = labels.loc[training_label_ids]
    
    assert training_images.shape[0] == training_labels.shape[0]
    return training_images, training_labels



In [44]:
images, labels = extract_training_data("Data/20171029-201949.h264.avi", "Data/20171029-201949.h264.csv", IMAGE_SIZE)

In [45]:
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (3, 3), kernel_initializer="he_normal", activation='relu'),
                          input_shape=(4, 64, 64, 3)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(32, return_sequences=False))
model.add(Dense(3, activation='linear'))

model.compile(loss="mse", optimizer="adam")
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_7 (TimeDist (None, 4, 62, 62, 32)     896       
_________________________________________________________________
time_distributed_8 (TimeDist (None, 4, 31, 31, 32)     0         
_________________________________________________________________
time_distributed_9 (TimeDist (None, 4, 30752)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                3940480   
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 99        
Total params: 3,941,475
Trainable params: 3,941,475
Non-trainable params: 0
_________________________________________________________________


In [50]:
print(labels.head())
y_train = labels.as_matrix(columns=labels.columns[1: -1])
#prediction = model.predict(images)

#for i in range(len(prediction)):
#    if i % 10 == 0:
#        print(np.mean(images[i], axis=(1, 2, 3)), prediction[i])

# for i in range(len(images)):
#     # print(np.mean(images[i], axis=(1,2,3)))
#     print(labels.head(10))

    Timestamp_s  Steering  Braking  Throttle  Gear
48     0.800033 -0.126390      0.0       1.0   2.0
60     1.000025  0.270588      0.0       1.0   2.0
72     1.200017  0.242210      0.0       1.0   2.0
84     1.400009  0.163224      0.0       1.0   1.0
96     1.600001 -0.268356      0.0       1.0   0.0


In [47]:
history = model.fit(images, y_train, batch_size=64, epochs=5, validation_split=0.04)

Train on 1671 samples, validate on 70 samples
Epoch 1/5
1671/1671 [==============================] - 36s - loss: 1.3678 - val_loss: 1.9821
Epoch 2/5
1671/1671 [==============================] - 35s - loss: 0.8371 - val_loss: 1.3662
Epoch 3/5
1671/1671 [==============================] - 37s - loss: 0.5049 - val_loss: 0.9732
Epoch 4/5
1671/1671 [==============================] - 36s - loss: 0.3181 - val_loss: 0.7183
Epoch 5/5
1671/1671 [==============================] - 38s - loss: 0.2148 - val_loss: 0.5489


In [48]:
images, _ = extract_training_data("Data/20171029-201639.h264.avi", "Data/20171029-201639.h264.csv", IMAGE_SIZE)

prediction = model.predict(images)

#for i in range(len(prediction)):
#    if i % 10 == 0:
#        print(np.mean(images[i], axis=(1,2,3)), prediction[i])

# #
# for i in range(len(images)):
#     print(np.mean(images[i], axis=(1,2,3)))

In [49]:
for i in range(len(prediction)):
    if i % 10== 0:
        print(prediction[i])

[ 0.10932445  0.00959995  1.40309608]
[ 0.10932445  0.00959995  1.40309608]
[ 0.10932445  0.00959995  1.40309608]
[ 0.10932445  0.00959995  1.40309608]
[ 0.10932445  0.00959995  1.40309608]
[ 0.10932445  0.00959995  1.40309608]
[ 0.10932445  0.00959995  1.40309608]
[ 0.10932445  0.00959995  1.40309608]
[ 0.10932445  0.00959995  1.40309608]
[ 0.10932445  0.00959995  1.40309608]
[ 0.10932445  0.00959995  1.40309608]
[ 0.10932445  0.00959995  1.40309608]
[ 0.10932445  0.00959995  1.40309608]
[ 0.10932445  0.00959995  1.40309608]
[ 0.10932445  0.00959995  1.40309608]
[ 0.10932445  0.00959995  1.40309608]
[ 0.10932445  0.00959995  1.40309608]
[ 0.10932445  0.00959995  1.40309608]
[ 0.10932445  0.00959995  1.40309608]
[ 0.10932445  0.00959995  1.40309608]
[ 0.10932445  0.00959995  1.40309608]
[ 0.10932445  0.00959995  1.40309608]
[ 0.10932445  0.00959995  1.40309608]
[ 0.10932445  0.00959995  1.40309608]
[ 0.10932445  0.00959995  1.40309608]
[ 0.10932445  0.00959995  1.40309608]
[ 0.10932445